# Extracting Text

In [1]:
import fitz  # PyMuPDF is imported as fitz

In [2]:
# Open the PDF document
def extract_text_from_pdf(pdf_file_path):
    test_doc = fitz.open(pdf_file_path)  # change the path to the pdf file

    # Define thresholds to extract headers and footers
    header_threshold = 0.15  # Top 15% of the page height
    footer_threshold = 0.925  # Bottom 7.5% of the page height

    ## EXTRACT THE TEXT BY BLOCKS

    # Initialize a variable to hold the extracted text
    extracted_text_block = []
    text_string = ""

    # Loop through each page
    for page_num in range(len(test_doc)):
        page = test_doc.load_page(page_num)  # Load the page
        text_block = page.get_text("blocks")  # Extract text from the page as blocks
        page_height = page.rect.height  # Get the height of the page

        for block in text_block:
            # Get the vertical position of the block (block[1] is the top y-coordinate, block[3] is the bottom y-coordinate)
            block_top = block[1]
            block_bottom = block[3]

            # Filter out blocks that are within the header or footer region
            if (block_top > page_height * header_threshold) and (
                block_bottom < page_height * footer_threshold
            ):
                text = block[
                    4
                ]  # The text content is in the 5th element of the block tuple
                text_string += text + "\n\n"  # Write the block text
    # Close the document
    test_doc.close()
    return text_string

In [3]:
original_minute = r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\data\2024\acta_30_07_2024\Acta Sesión Ordinaria  No. 076 2024-07-30.pdf"

extracted_minute = extract_text_from_pdf(original_minute)
print(extracted_minute)

SESIÓN ORDINARIA DE CONCEJO No. 076 DE 30 DE JULIO DE 2024 


 


SUMARIO: 


CAPÍTULOS   
 
 
TEMA 


 


I. 
Verificación del cuórum. 


 


II. 
Instalación de la sesión. 


 


III. 
Lectura del Orden del Día.   


 


IV. 
Himno a Quito.  


 


V. 
Comisiones Generales: 1. Comisión General para recibir a la Dra. 
Verónica Minaya, Oficial del Sector Ciencias Naturales de la 
Oficina Regional de la UNESCO en Quito, representante para 
Bolivia, Colombia, Ecuador y Venezuela, a fin de que exponga ante 
el Concejo Metropolitano sobre la importancia del fortalecimiento 
institucional para la gestión y conservación de la "Reserva de 
Biósfera del Chocó Andino de Pichincha" (Oficio Nro. GADDMQ-
AM-2024-1474-OF). 


 


VI. 
Comisión General para recibir al señor Ynti Felipe Arcos Torres, en 
representación del Colectivo Quito Sin Minería-Cabildo Cívico de 
Quito, a fin de que exponga ante el Concejo Metropolitano sobre la 
problemática social y ambiental que atraviesa el Chocó Andino 
(O

# Connection to OpenAI API

In [4]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [5]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

##CHANGE EVERYTIME THE PATH
extracted_raw_data = extracted_minute
text_content = []

# Count the number of tokens    
text_tokens = encoding.encode(extracted_raw_data)
num_tokens = len(text_tokens)
print(f"Number of tokens : {num_tokens}")

text_content.append(extracted_raw_data) #save for future processing
     
print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 65345

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [6]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [6]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]
client = OpenAI(api_key = key_ct)

### Prompt Engineering (Zero-shot)

### One approach

In [ ]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
full_instruction = """Has un resumen de una minuta de reunión de un gobierno local con las siguientes instrucciones:
-  Enfatiza Los principales temas que se hablaron en la sesion, problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan cuando sea necesario.
    - Provee datos concretos
    - No emitas juicios de valor
    - Pon titulos a las secciones de acuerdo a los temas tratados,simplificando y unificando las secciones si es necesario
  -   Envía la información con la siguiente estructura{
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesion (nombre) No. (número de la sesion)", 
     "content:  [" resumen de la sesion en donde se explique de manera fluida y clara con las instrucciones enviadas arriba"], 
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


### CoT approach

In [7]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
full_instruction = """Eres un asistente que resume minutas de reunión de un gobierno local razonando a traves de los siguientes pasos:
Paso 1: Identifica en el indice los temas principales tratados en la sesion
Paso 2: Resume cada tema en un parrafo de 3-5 oraciones, describe problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan cuando sea necesario.
Paso 3: Unifica los temas que tengan relación en una sección.
Paso 4: Coloca titulos en cada sección de acuerdo a los temas tratados. No pongas ninguna numeración.
Paso 4: Escribe el resumen general de la sesión con fluencia y claridad
Paso 5: Envía la información con la siguiente estructura{
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesion (nombre) No. (número de la sesion)", 
     "content:  [" resumen de la sesion con titulos y secciones"], 
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 231


In [8]:

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : full_instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.80,
    max_tokens = 1200,
)


### Sending instructions

In [9]:

response_summary_vers001 = response.choices[0].message.content
print(response_summary_vers001)   

{
     "date" : "2024-07-30", 
     "document_id" : "acta_076", 
     "act_name": "Sesion Ordinaria de Concejo No. 076", 
     "content": [
          "Verificación del Cuórum",
          "La sesión se inició con la verificación del cuórum, confirmando la presencia de 19 concejales, lo que permitió dar inicio a la sesión ordinaria del Concejo Metropolitano de Quito, presidida por el Alcalde Pabel Muñoz López. ",
          "Comisiones Generales: Exposición sobre la Reserva de Biósfera del Chocó Andino",
          "La Dra. Verónica Minaya, representando a la UNESCO, presentó la importancia de la Reserva de Biósfera del Chocó Andino y la necesidad de fortalecer la gestión institucional para su conservación. Destacó cómo la reserva no solo protege la biodiversidad, sino que también promueve el desarrollo sostenible y la participación comunitaria. El Alcalde y varios concejales se mostraron comprometidos con la causa.",
          "Problemáticas del Chocó Andino",
          "El señor Ynti Fel

### Opcional JSON formatting

In [ ]:

# #extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_vers001 = extract_markdown_content(response_summary_vers001)
print(response_summary_vers001)


# MongoDB Connection

In [11]:
import json
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv(override=True)
db_url = os.environ["DATABASE_URL"]

In [ ]:
client = MongoClient(db_url)
db = client["cv_tech"]
collection = db["Summary"]
client.admin.command('ping')

In [12]:
response_summary_dict = json.loads(response_summary_vers001)

In [ ]:

# Convert the content to a dictionary
try:
    response_summary_dict = json.loads(response_summary_vers001)
    # Add a new key-value pair to the dictionary - CHANGE EVERYTIME
    response_summary_dict["document_url"] = "https://www7.quito.gob.ec/mdmq_ordenanzas/Administraci%C3%B3n%202023-2027/Actas%20Sesiones/2024/ACTA%2076%20SESIO%CC%81N%20ORDINARIA%20DE%2030%20DE%20JULIO%20DE%202024-1-signed.pdf"

    print(response_summary_dict)
    
    
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

### Insert to MongoDB collection

In [ ]:
try:
    result = collection.insert_one(response_summary_dict)
    print(f"Inserted document ID: {result.inserted_id}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

### Save copy to JSON data file

In [13]:

# Write the JSON data to a new file
#CHANGE THIS FILE NAME
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\data\json_files\c_o_t_example", "w", encoding="utf-8") as json_file: #CHANGE THIS FILE NAME
    json.dump(response_summary_dict, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")

Text extraction completed and saved as JSON.
